In [2]:
import os
import sys

os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.6 (default, Oct 26 2016 20:30:19)
SparkSession available as 'spark'.


In [3]:
#import os
#import sys


#os.environ['SPARK_HOME'] = '/usr/lib/spark'
#sys.path.insert(0, '/usr/lib/spark/python/lib/py4j-0.9-src.zip')
#sys.path.insert(0, '/usr/lib/spark/python/')
#sys.path.insert(0, '/usr/local/lib64/python2.7/site-packages')
#sys.path.insert(0,'/usr/local/lib/python2.7/site-packages')

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import seaborn as sns

/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [6]:
import numpy as np
import pandas as pd
import time
import json

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import pyspark.sql.functions as F

In [7]:
from operator import add
from pyspark.sql.types import ArrayType, MapType

# Summary

## Train
* total items 142'437 (single row is filtered out)
* number of recommended items 
  * min 3, max >100, avg ~6, stddev ~5
* score of recommended item
  * min 1, max >100, avg ~3, stddev ~3
* has outlier line with 40K recommendations - it is filtered out
* There are 337'756 unique items among recomended
* There is no duplicates in trai set


## Test
* total items 60'956
* 47'932 test items occured at least single time as recommended to some item in the train set
* number of backreference to the train set: min=0, 25% = 1, median = 2, avg = 3, max=51

## Active items
* There are 380'900 items in train + train_rec + test set
* train, train_rec	112'317
* train             30'120
* train_rec     	177'507
* test  	        13'024
* test, train_rec	47'932

## Item description
* Description json is available for 9'273'625 items
* **parent_id**	is available for 3'805'884
* there are 80 custom attributes
* **attr1** looks like a **name**, and available for 9'273'137
* **attr0** looks like a **description**, and available for 8'272'266
* There are 5842 items with duplicated descriptions within Activel Items

## Ratings
* Roughly speaking is available in 20-30% of active items


# Train set

In [17]:
items_raw = sc.textFile("/labs/project2//ozon_train.txt")

In [18]:
items_raw = items_raw.map(lambda x: json.loads(x))

In [19]:
#train_items = items_raw.map(lambda x: (x['item'], 
#                                       x['true_recoms'], 
#                                       len(x['true_recoms']), 
#                                       float(np.mean(x['true_recoms'].values()))))
#train_items.cache()

In [20]:
train_items_narrow = items_raw.map(lambda x: (x['item'], x['true_recoms']))\
                              .filter(lambda x: len(x[1])<40000)\
                              .toDF()\
                              .selectExpr("_1 as item", "_2 as rec_items")\
                              .select(F.col("item"), F.explode(F.col("rec_items")).alias("rec_item","clicks"))

In [21]:
train_items_narrow.show(3)

+--------+--------+------+
|    item|rec_item|clicks|
+--------+--------+------+
|31471303|31471373|     1|
|31471303|31471307|     3|
|31471303|31471342|     1|
+--------+--------+------+
only showing top 3 rows



# Test set

In [22]:
test = spark.read.json("/labs/project2//ozon_test.txt")

In [23]:
test.show(2)

+--------+------+
|    item|recoms|
+--------+------+
|28759795|  null|
|32712593|  null|
+--------+------+
only showing top 2 rows



In [24]:
test.count()

60956

# Item details

In [25]:
item_descr_raw = sc.textFile("/labs/project2//item_details_full")

In [26]:
items_descr_df = item_descr_raw.map(lambda x: json.loads(x))\
                          .map(lambda x: (x['itemid'], 
                                          x.get('attr1',None), 
                                          x.get('attr0',None),
                                          x.get('parent_id',None),
                                          x.get('attr53',None),
                                          x.get('attr2',None),
                                          x.get('attr32',None),
                                          x.get('attr18',None),
                                          ))\
                          .toDF(schema=StructType([
                                        StructField("itemid", StringType()),
                                        StructField("name", StringType()),
                                        StructField("descr", StringType()),
                                        StructField("parent_id", StringType()),
                                        StructField("a53_isbn", StringType()),
                                        StructField("a02_author", StringType()),
                                        StructField("a32_lang", StringType()),
                                        StructField("a18_publisher", StringType()),
                                       ]))

In [27]:
items_descr_df.show(3)

+--------+--------------------+--------------------+---------+--------+--------------+---------+--------------------+
|  itemid|                name|               descr|parent_id|a53_isbn|    a02_author| a32_lang|       a18_publisher|
+--------+--------------------+--------------------+---------+--------+--------------+---------+--------------------+
|12808031|История маленьког...|<B> Прижизненное ...| 17969540|    null|   Ф. Бернет  |Русский  |Издание А. Ф. Дев...|
|12599476|Belkin чехол для ...|Надежная защита -...| 16095306|    null|          null|     null|                null|
|22440986|Толковый словарь ...|               нет  |     null|    null|С. И. Ожегов  |     null|                null|
+--------+--------------------+--------------------+---------+--------+--------------+---------+--------------------+
only showing top 3 rows



In [28]:
#items_descr_df.filter(~F.isnull(F.col("attr53"))).show()

# Ratings

In [31]:
item_ratings = spark.read.json("/labs/project2/ratings", 
                              schema=StructType([
            StructField("itemid", StringType()),
            StructField("rating", FloatType())
        ]))

In [32]:
item_ratings.show(2)

+-------+------+
| itemid|rating|
+-------+------+
|2515646|   5.0|
|2858646|   5.0|
+-------+------+
only showing top 2 rows



# Catalogs

In [33]:
item_catalogs = spark.read.json("/labs/project2/catalogs")

In [34]:
item_catalogs.show(3)

+---------+--------+
|catalogid|  itemid|
+---------+--------+
|  1139010|26881367|
|  1140991|31587713|
|  1134295|17425282|
+---------+--------+
only showing top 3 rows



# Catalogs structure (path)

In [35]:
catalog_path = spark.read.json("/labs/project2/catalog_path", 
                               schema=StructType([
                                            StructField("catalogid", StringType()),
                                            StructField("catalogpath", 
                                                        ArrayType(MapType(StringType(), StringType())))
                                       ]))

In [36]:
catalog_path.show(2, truncate=False)

+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|catalogid|catalogpath                                                                                                                                                                                                                   |
+---------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1125369  |[Map(1125369 -> Аксессуары), Map(1123855 -> Коты), Map(1123646 -> Я_Символ года 2011), Map(1110181 -> Я_ Новогодняя тематика), Map(1106018 -> ПОДАРКИ. СЕЗОННЫЕ ПРЕДЛОЖЕНИЯ), Map(1134031 -> ПОДАРКИ_СТАРЫЕ ВЕТКИ_НЕ АКТИВНЫ)]|
|1125370  |[Map(1125370 -> Магнитики), Map(1123855 -> Коты),

# Register all tables

In [37]:
train_items_narrow.registerTempTable("train_narrow")
test.registerTempTable("test")
#items_descr_df.registerTempTable("descr")
item_ratings.registerTempTable("rating")
item_catalogs.registerTempTable("catalogs")
catalog_path.registerTempTable("catalog_path")


In [38]:
item_clicks = spark.sql("""select rec_item as itemid,
                                 count(*) as n_recomended,
                                 sum(clicks) as ttl_clicks
                          from train_narrow
                          group by rec_item
                       """)

In [39]:
item_clicks.show(2)

+--------+------------+----------+
|  itemid|n_recomended|ttl_clicks|
+--------+------------+----------+
|31390826|           5|        23|
| 5621791|           3|        10|
+--------+------------+----------+
only showing top 2 rows



In [40]:
item_clicks.count()

337756

In [41]:
item_clicks.registerTempTable("clicks")

In [42]:
test_with_catalogs = spark.sql("""select 
                                    t.item as itemid,
                                    catalogid
                                 from test t 
                                 left join catalogs on catalogs.itemid = t.item
                              """)

In [43]:
test_with_catalogs.registerTempTable("test_with_catalog")

In [44]:
tcs=spark.sql("""select 
                itemid,sum(case when catalogid is null then 0 else 1 end) as n_catalogs
            from test_with_catalog
            group by itemid
         """)

In [45]:
tcs_df = tcs.toPandas()

In [46]:
#tcs_df.n_catalogs.value_counts()

In [47]:
tcs_df[tcs_df.n_catalogs==0].head(3)

,itemid,n_catalogs
58,25677428,0
90,28128873,0
118,30306599,0


In [48]:
#items_descr_df.filter(F.col("itemid") == "20999092").show(truncate=False)
#has no parent id, no catalogid

In [49]:
test_catalogs_freq=spark.sql("""select 
                catalogid,count(*) as n_test_records
            from test_with_catalog
            group by catalogid
         """)

In [50]:
test_catalogs_freq_df = test_catalogs_freq.toPandas()

In [51]:
test_catalogs_freq_df.sort_values(by="n_test_records", ascending=False).head(5)

,catalogid,n_test_records
2304,1141109,2224
737,None,1634
3294,1145568,787
287,1146150,738
980,1158883,726


In [52]:
test_catalogs_freq.registerTempTable("catalogs_in_test")

In [53]:
# Pivot based on catalog+rating+click using only catalogs from test

In [54]:
q = spark.sql("""
        select 
            c.itemid,
            c.catalogid,
            r.rating,
            cl.ttl_clicks,
            cl.n_recomended,
            cl.ttl_clicks * 1.0 / cl.n_recomended  as avg_clicks
        from catalogs c
        join catalogs_in_test ct on ct.catalogid = c.catalogid
        left join rating r on c.itemid = r.itemid
        left join clicks cl on c.itemid = cl.itemid
""")

In [55]:
q.registerTempTable("pivot_item_in_test_catalogs")

In [56]:
q.show(2)

+------+---------+------+----------+------------+----------+
|itemid|catalogid|rating|ttl_clicks|n_recomended|avg_clicks|
+------+---------+------+----------+------------+----------+
|100010|  1142025|   3.0|      null|        null|      null|
|100140|  1139877|  null|      null|        null|      null|
+------+---------+------+----------+------------+----------+
only showing top 2 rows



In [57]:
test_catalog_items_summary = spark.sql("""
        select
           catalogid,
           count(*) as n_items,
           sum(case when rating is not null then 1 else 0 end) as n_ratings,
           sum(case when avg_clicks is not null then 1 else 0 end) as n_from_rec,
           avg(rating) as avg_rating,
           avg(ttl_clicks) as avg_ttl_clicks,
           avg(avg_clicks) as avg_clicks_per_rec
        from pivot_item_in_test_catalogs
        group by catalogid
""")

In [58]:
test_cats_sum_df = test_catalog_items_summary.toPandas()

In [59]:
test_cats_sum_df.sort_values(by='n_items', ascending=False).head(5)

,catalogid,n_items,n_ratings,n_from_rec,avg_rating,avg_ttl_clicks,avg_clicks_per_rec
1679,1138615,1414978,549,1096,3.982046,3.493613,2.27689719799665055139507
305,1134295,712106,71,3560,4.450704,2.925000,1.73293370019493614999171
666,1138452,273724,212,460,3.598334,3.032609,2.08360248447204968944109
2301,1141109,226980,20562,13431,4.264201,8.209069,3.14087727695402387723821
68,1134294,192024,58,1538,4.678161,3.539662,1.70859294224053651881918


In [60]:
#spark.sql("select * from catalog_path where catalogid in ('1169813','1178624')").show(truncate=False)

In [61]:
test_cats_sum_df.sort_values(by='n_items', ascending=False).tail(5)

,catalogid,n_items,n_ratings,n_from_rec,avg_rating,avg_ttl_clicks,avg_clicks_per_rec
955,1177090,3,2,3,4.416667,13.333333,2.67777777777777777780000
2159,1167240,3,0,0,NaN,NaN,None
904,1128816,2,0,0,NaN,NaN,None
3029,1169813,2,0,0,NaN,NaN,None
2058,1158590,2,0,2,NaN,17.000000,9.33333333333333333335000


In [62]:
items_with_catalog_rank = spark.sql("""
        select
           *,
           row_number() over (partition by catalogid order by rating desc,n_recomended desc,ttl_clicks desc) catalog_rank
        from pivot_item_in_test_catalogs 
""")

In [63]:
items_with_catalog_rank.show(2)

+-------+---------+------+----------+------------+----------+------------+
| itemid|catalogid|rating|ttl_clicks|n_recomended|avg_clicks|catalog_rank|
+-------+---------+------+----------+------------+----------+------------+
|3380321|  1102913|   5.0|      null|        null|      null|           1|
|4129421|  1102913|   1.0|      null|        null|      null|           2|
+-------+---------+------+----------+------------+----------+------------+
only showing top 2 rows



In [64]:
#!!!!!!
#items_with_catalog_rank.write.parquet("lab10/items_with_catalog_rank")

In [65]:
items_with_catalog_rank.registerTempTable("items_with_catalog_rank")

In [66]:
test_catalog_items_summary.registerTempTable("test_catalog_items_summary")

In [67]:
top100per_cat_1 = spark.sql("""
       select 
           t.itemid as test_itemid,
           t.catalogid as test_catalogid,
           ir.*,
           cs.n_items,
           cs.n_ratings,
           cs.n_from_rec,
           cs.avg_rating,
           cs.avg_ttl_clicks
       from test_with_catalog t
       join items_with_catalog_rank ir on t.catalogid=ir.catalogid and t.itemid != ir.itemid
       join test_catalog_items_summary cs on t.catalogid = cs.catalogid
       where ir.catalog_rank <= 102
""")

In [68]:
top100per_cat_1.show(2)

+-----------+--------------+-------+---------+------+----------+------------+----------+------------+-------+---------+----------+----------+-----------------+
|test_itemid|test_catalogid| itemid|catalogid|rating|ttl_clicks|n_recomended|avg_clicks|catalog_rank|n_items|n_ratings|n_from_rec|avg_rating|   avg_ttl_clicks|
+-----------+--------------+-------+---------+------+----------+------------+----------+------------+-------+---------+----------+----------+-----------------+
|   32111279|       1102913|3380321|  1102913|   5.0|      null|        null|      null|           1|    271|        2|         7|       3.0|5.142857142857143|
|   32111279|       1102913|4129421|  1102913|   1.0|      null|        null|      null|           2|    271|        2|         7|       3.0|5.142857142857143|
+-----------+--------------+-------+---------+------+----------+------------+----------+------------+-------+---------+----------+----------+-----------------+
only showing top 2 rows



In [69]:
top100per_cat_1.printSchema()

root
 |-- test_itemid: string (nullable = true)
 |-- test_catalogid: string (nullable = true)
 |-- itemid: string (nullable = true)
 |-- catalogid: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- ttl_clicks: long (nullable = true)
 |-- n_recomended: long (nullable = true)
 |-- avg_clicks: decimal(38,19) (nullable = true)
 |-- catalog_rank: integer (nullable = true)
 |-- n_items: long (nullable = false)
 |-- n_ratings: long (nullable = true)
 |-- n_from_rec: long (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- avg_ttl_clicks: double (nullable = true)



In [70]:
top100per_cat_1.registerTempTable("top100per_cat_1")

In [71]:
res_cat01 = spark.sql("""
        select
           test_itemid,
           itemid,
           catalogid,
           rating as rec_item_rating,
           ttl_clicks as rec_item_clicks,
           n_recomended as rec_item_n_recomended,
           row_number() over (partition by test_itemid order by n_items desc, catalog_rank) cat01_rank
        from top100per_cat_1 
""")

In [72]:
res_cat01.show(2)

+-----------+--------+---------+---------------+---------------+---------------------+----------+
|test_itemid|  itemid|catalogid|rec_item_rating|rec_item_clicks|rec_item_n_recomended|cat01_rank|
+-----------+--------+---------+---------------+---------------+---------------------+----------+
|   13078902|28044846|  1139642|            5.0|              7|                    3|         1|
|   13078902|19630709|  1139642|            5.0|             14|                    2|         2|
+-----------+--------+---------+---------------+---------------+---------------------+----------+
only showing top 2 rows



In [73]:
res_cat01.write.parquet("res_cat01")

In [74]:
def merge_dicts(x, y):
    x.update(y)
    return x

In [75]:
q=res_cat01.filter("cat01_rank<=100").select("test_itemid", F.create_map(["itemid", "cat01_rank"]).alias("iwr"))\
          .rdd.reduceByKey(merge_dicts).cache()

In [76]:
res_cat01_list = q.collect()

In [77]:
res_cat01_dict = {r[0]:{k:1000.0/v for k,v in r[1].items()} for r in res_cat01_list}

In [78]:
len(res_cat01_dict.keys())

59322

In [89]:
#res_cat01_dict['32486081']

In [85]:
with open("Data/ozon_test.txt") as inh, open("project02.txt", 'w') as outh:
    for line in inh.readlines():
        tr = json.loads(line)
        tr["recoms"]=res_cat01_dict.get(tr["item"], [])
        ln=json.dumps(tr)
        outh.write(ln)
        outh.write("\n")

In [107]:
#items_descr_df.filter(F.col("itemid").isin("13078902","28044846","19630709")).show(truncate=False)

In [90]:
##
test_in_train = spark.sql("""
      select 
          t.item as test_item,
          tn.item as item,
          sum(clicks) as ttl_clicks,
          count(*) as n_recomended
      from test t
      join train_narrow tn on t.item = tn.rec_item
      group by t.item, tn.item
""")

In [91]:
q2=test_in_train.withColumn("tit_score", F.col("ttl_clicks") * F.lit(100000) + F.lit(1000)*F.col("n_recomended"))

In [92]:
q2m=q2.select("test_item", F.create_map(["item", "tit_score"]).alias("iwr"))\
          .rdd.reduceByKey(merge_dicts).cache()

In [93]:
q2m.take(1)

[(u'33137403', {u'33137418': 101000, u'33137423': 101000})]

In [94]:
res02_tit_list = q2m.collect()

In [95]:
res02_tit_dict = {r[0]:{k:v for k,v in r[1].items()} for r in res02_tit_list}

In [96]:
x0 = res_cat01_dict['32486081']

In [97]:
x1 = res02_tit_dict['32486081']

In [98]:
x1

{u'30741894': 101000,
 u'31738656': 101000,
 u'31738732': 101000,
 u'32486280': 101000,
 u'32486952': 1401000,
 u'33213035': 1101000}

In [99]:
#item_catalogs.filter(F.col("itemid").isin("32486081","33213035","32486952")).show(truncate=False)

In [100]:
def merge_ranks(x1,x2, ncut=100):
    k1 = set(list(x1.keys()))
    k2 = set(list(x2.keys()))
    kall = k1 | k2
    res = []
    for k in kall:
        v1 = x1.get(k, 0)
        v2 = x2.get(k, 0)
        res.append((k,v1+v2))
    res = sorted(res, key=lambda q: (-q[1],q[0]))
    n = min(ncut, len(res))
    return {k:v for k,v in res[:n]}

In [102]:
with open("Data/ozon_test.txt") as inh, open("project02.txt", 'w') as outh:
    for line in inh.readlines():
        tr = json.loads(line)
        x0 = res_cat01_dict.get(tr["item"], {})
        x1 = res02_tit_dict.get(tr["item"], {})
        tr["recoms"] = merge_ranks(x1, x0, 50)
        ln=json.dumps(tr)
        outh.write(ln)
        outh.write("\n")

In [103]:
!cat project02.txt | head -n10

{"item": "28759795", "recoms": {"26098283": 83.33333333333333, "26098282": 55.55555555555556, "28759807": 250.0, "21014430": 28.571428571428573, "19381548": 166.66666666666666, "28759803": 202000.0, "28015795": 30.303030303030305, "21014425": 27.027027027027028, "28759796": 101000, "19381466": 41.666666666666664, "28015796": 22.727272727272727, "28759793": 32.25806451612903, "28759792": 201014.08450704225, "7020767": 52.63157894736842, "30213204": 500.0, "28759789": 33.333333333333336, "5581444": 24.390243902439025, "4998555": 111.11111111111111, "7020508": 47.61904761904762, "31594275": 22.22222222222222, "4899191": 43.47826086956522, "20344327": 71.42857142857143, "32662317": 23.80952380952381, "20344349": 38.46153846153846, "24326890": 142.85714285714286, "20767160": 35.714285714285715, "28188636": 23.25581395348837, "24326898": 37.03703703703704, "24326899": 100.0, "21494479": 58.8235294117647, "7020565": 27.77777777777778, "26098279": 45.45454545454545, "31114007": 40.0, "14648405